In [1]:
#神经网络
#可以使用torch.nn包来构建神经网络
#我们已经介绍了autograd，nn包则依赖于autograd包来定义模型并对它们求导
#一个nn.Model包含各个层和一个forward(input)方法，该方法返回output。

#一个神经网络会接受一个参数，然后将它送入下一层，一层接一层的传递，最后给出输出
#一个神经网络的典型训练过程如下：
#--定义包含一些可学习参数的神经网络
#--在输入数据集上进行迭代
#--通过网络处理输入
#--计算损失(输出和正确答案的距离)
#--将梯度反向传播给网络的参数
#--更新网络的权重，一般使用一个简单的规则：weight = weight - learning_rate*gradient








In [5]:
#定义网络
#让我们定义这样一个网络

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        #输入图像channel:1；输出channel:6,5*5的卷积核
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6,16,5)
        #线性操作:y = Wx + b
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        #2*2的max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        #如果是方阵，可以只使用一个数字进行定义
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
    
        return x

    def num_flat_features(self, x):
        size = x.size()[1:] #除去批处理的维度，留下其他的维度
        num_features = 1
        
        for s in size:
            num_features *= s
        return num_features
    

net = Net()
print(net)
    
    
#我们只需要定义forward函数，backward函数会在使用autograd时自动定义，backward函数用来计算导数
#可以在forward函数中使用任何针对张量的操作和计算
    
#一个模型的可学习参数可以通过net.parameters()返回
params = list(net.parameters())
print(len(params))
print(params[0].size())#conv1的参数
    
    
    
    




Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
10
torch.Size([6, 1, 5, 5])


In [9]:
#让我们尝试一个随机的32*32的输入

input = torch.randn(1,1,32,32)
out = net(input)
print(out)

#再清除所有参数的梯度缓存，然后进行随机梯度的反向传播
net.zero_grad()
out.backward(torch.randn(1,10))

#注意torch.nn只支持小批量处理(mini-batches)。整个torch.nn包只支持小批量样本的输入，不支持单个样本
#比如，nn.Conv2d接受一个四维的张量，即nSamples * nChannels * Height * Width
#如果是一个单独的样本，只需要使用input.unsqueeze(0)来添加一个假的批大小维度

#autograd.Function:实现了自动求导前向和反向传播的定义，每个Tensor至少创建一个Function节点，
#该节点连接到创建Tensor的函数并对其历史进行编码



tensor([[-0.1289, -0.0417,  0.0084,  0.1442,  0.0124,  0.0107,  0.1236, -0.0420,
          0.0785, -0.0832]], grad_fn=<AddmmBackward>)


In [10]:
#a = torch.rand(2,3) rand()随机产生【0,1】之间的随机数
#b = torch.randn(2,3),randn()随机产生高斯分布
#print(a)
#print(b)

In [24]:
#损失函数
#一个损失函数接受一对(output, target)作为输入，计算一个值来估计网络的输出与目标值相差多少
#nn包中有很多不同的损失函数，nn.MSELoss【mean-squared error】是比较简单的一种，它计算输出和目标的均方误差

output = net(input)
target = torch.randn(10)
target = target.view(1,-1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

#接下来使用loss的.grad_fn属性跟踪方向传播过程，会看到计算图如下
#input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
# -> view -> linear -> relu -> linear -> relu -> linear
# -> MSELoss
# -> loss

#所以，当我们调用loss.backward()，整张图开始关于loss微分，图中所有设置了requires_grad = True的张量的.grad属性累积着梯度张量
#为了说明这一点，我们可以向后跟踪几步

print(loss.grad_fn) #MSELoss
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])


tensor(1.6794, grad_fn=<MseLossBackward>)


In [25]:
#反向传播

#我们只需要调用loss.backward()来反向传播权重。我们需要清零现有的梯度，否则梯度将会与已有的梯度累加
#现在，我们将调用loss.backward()，并查看conv1层的偏置(bias)在反向传播前后的梯度

net.zero_grad()#清除所有参数(parameter)的梯度缓存

print("conv1.bias.grad before backward:", net.conv1.bias.grad)

loss.backward()

print("conv1.bias.grad after backward:", net.conv1.bias.grad)







conv1.bias.grad before backward: tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward: tensor([-0.0119, -0.0137,  0.0020,  0.0008,  0.0007,  0.0120])


In [26]:
#更新权重
#最简单的更新规则就是随机梯度下降法(SGD)
#weight = weight - learning_rate*gradient
#可以使用简单的python代码来实现：
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)





In [27]:
#跟新规则有SGD, Nesterov-SGD, Adam, RMSProp等，
#因此有一个较小的包torch.optim，他实现了所有的这些方法，使用它很简单
import torch.optim as optim

#创建优化器(optimizer)
optimizer = optim.SGD(net.parameters(), lr=0.01)

#在训练迭代过程中：
optimizer.zero_grad()  #清除梯度缓存
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()







